In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from keras.preprocessing import image 

import tensorflow as tf
import os
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Dropout, Flatten, Dense
from tensorflow.keras import Model
from sklearn.utils import shuffle

Using TensorFlow backend.


In [2]:
name = ['eye_color', 'face_shape']
#data = pd.read_csv('./dataset_AMLS_20-21/celeba/labels.csv',sep='\t',header=0,names = name)
data = pd.read_csv('./dataset_AMLS_20-21/cartoon_set/labels.csv',sep='\t')
Y_A10=data['eye_color']
Y_A11=data['face_shape']

In [3]:
#图片读取
train_image = [] 
for i in tqdm(range(data.shape[0])):
    #img = image.load_img('./dataset_AMLS_20-21/celeba/img/'+str(i)+'.jpg', color_mode='rgb', target_size=None, grayscale=False)
    img = image.load_img('./dataset_AMLS_20-21/cartoon_set/img/'+str(i)+'.png', target_size=None, grayscale=True)
    img = image.img_to_array(img)     
    img = img/255     
    train_image.append(img) 
X = np.array(train_image)

100%|██████████| 10000/10000 [00:36<00:00, 274.62it/s]


In [4]:
X, Y_A10 = shuffle(X,Y_A10)
x_train, x_test, y_train, y_test = train_test_split(X, Y_A10,random_state=0)
print(x_train.shape,x_test.shape,y_train.shape)

(7500, 500, 500, 1) (2500, 500, 500, 1) (7500,)


In [5]:
# y_train = y_train.astype(np.uint8)
# y_test = y_test.astype(np.uint8)
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()
x_train = x_train/255
x_test = x_test/255
# x_train = x_train.astype(np.uint8)/255
# x_test = x_test.astype(np.uint8)/255
print(type(x_test),type(y_train))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [8]:
from tensorflow import keras

In [9]:
class Baseline(Model):
    def __init__(self):
        super(Baseline, self).__init__()
        self.c1 = Conv2D(filters=6, kernel_size=(40, 40), padding='same')  # 卷积层
        self.b1 = BatchNormalization()  # BN层
        self.a1 = Activation('relu')  # 激活层
        self.p1 = MaxPool2D(pool_size=(2, 2), strides=2, padding='same')  # 池化层
        self.d1 = Dropout(0.2)  # dropout层

        self.flatten = Flatten()
        self.f1 = Dense(5000, activation='relu')
        self.d2 = Dropout(0.2)
        self.f2 = Dense(5, activation='softmax')

    def call(self, x):
        x = self.c1(x)
        x = self.b1(x)
        x = self.a1(x)
        x = self.p1(x)
        x = self.d1(x)

        x = self.flatten(x)
        x = self.f1(x)
        x = self.d2(x)
        y = self.f2(x)
        return y


model = Baseline()

opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['sparse_categorical_accuracy'])

# checkpoint_save_path = "./checkpoint/Baseline.ckpt"
# if os.path.exists(checkpoint_save_path + '.index'):
#     print('-------------load the model-----------------')
#     model.load_weights(checkpoint_save_path)

# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_save_path,
#                                                  save_weights_only=True,
#                                                  save_best_only=True)

# #history = model.fit(x_train, y_train, batch_size=32, epochs=15, validation_data=(x_test, y_test), validation_freq=1,
#                     callbacks=[cp_callback])

history = model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_test, y_test), validation_freq=1)

model.summary()

# print(model.trainable_variables)
file = open('./weights.txt', 'w')
for v in model.trainable_variables:
    file.write(str(v.name) + '\n')
    file.write(str(v.shape) + '\n')
    file.write(str(v.numpy()) + '\n')
file.close()

###############################################    show   ###############################################

# 显示训练集和验证集的acc和loss曲线
acc = history.history['sparse_categorical_accuracy']
val_acc = history.history['val_sparse_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

Train on 7500 samples, validate on 2500 samples
Epoch 1/5
  32/7500 [..............................] - ETA: 49:55

ResourceExhaustedError:  OOM when allocating tensor with shape[32,500,500,6] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node baseline_1/conv2d_1/BiasAdd (defined at C:\Users\93554\anaconda3\envs\mytf\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_distributed_function_2683]

Function call stack:
distributed_function
